# Code examples for interacting with Mondo using OAK

### OAK
OAK, the Ontology Access Kit, is a Python library for common ontology operations and provides a variety of adapters to connect to ontologies. More information is available at: https://incatools.github.io/ontology-access-kit/index.html

### Create a Python virtual environment or new Conda environment
To avoid possible package dependency issues, create a new env to use this notebook. See [Getting Started](https://incatools.github.io/ontology-access-kit/intro/tutorial01.html#part-1-getting-started) with OAK for a reminder on how to create a Python environment or the [Conda Cheat Sheet](https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf) for commands to create a new conda environment.

### Install Python packages needed for this notebook

In [ ]:
# Install Python packages
# !pip install oaklib

In [ ]:
# Test OAK Installation
# !runoak --help

### Load Imports

In [1]:
# Load Imports

from pathlib import Path
from oaklib import get_adapter
from oaklib.mappers import OntologyMetadataMapper
from oaklib.datamodels.vocabulary import IS_A, PART_OF, SEMAPV

### Download a SQLite version of your ontology of interest
There are a few options to download a SQLite version of an ontology file to use with oaklib. The examples below will demonstrate how to use an additional Python package named SemSQL (another of the INCA Tools along with OAK) and using wget. 

More information on how to download a SQLite ontology file can be found here: https://incatools.github.io/ontology-access-kit/intro/tutorial07.html#download-a-sqlite-file

### SemSQL Option

In [ ]:
# SemSQL Option to download SQLite version of Mondo
# See: https://github.com/INCATools/semantic-sql#download-ready-made-sqlite-databases

# Install package
!pip install -q semsql

# Create download directory
tmp_directory = "./tmp"
ontology_db = "mondo.db"
ontology = "mondo"

# Fetch Mondo and test access to the SQLite version of the ontology
!semsql download $ontology -o $tmp_directory/$ontology_db

# Check that database file exists
!ls -ltr $tmp_directory

### wget Option

In [ ]:
# # Wget Option to download SQLite version of Mondo - Part 1

# # See https://incatools.github.io/ontology-access-kit/howtos/validate-an-obo-ontology.html#step-1-obtain-the-sqlite-version-of-the-ontology
# # for information on how to download a SQLite version of Mondo

# # Manually find file of interest from: https://s3.amazonaws.com/bbop-sqlite/
# filename = 'mondo.db.gz'
# url = f"https://s3.amazonaws.com/bbop-sqlite/{filename}"

# # Create download directory
# download_dir = './ontology_downloads'
# !mkdir -p $download_dir

# # Use wget to download the file to the specified directory
# !wget -q $url -P $download_dir && echo "Download complete!"

In [ ]:
# # Wget Option - Part 2
# # Use the unzip command to extract the contents of the zip file
# zip_filepath = f"{download_dir}/{filename}"

# ontology_db_file = "mondo.db"
# unzipped_filepath = f"{download_dir}/{ontology_db_file}"

# # Use gunzip to extract the contents of the gzipped file
# !gunzip -d -c $zip_filepath > $unzipped_filepath

# # Check that database file exists
# !ls -ltr $download_dir

### OAK CLI Option

In [ ]:
# # OAK CLI Option 
# !runoak -i sqlite:obo:mondo ontology-metadata mondo

# # NOTE: PyStow is used to cache the file, and the default location is ~/.data/oaklib.
# # https://incatools.github.io/ontology-access-kit/intro/tutorial07.html#fetching-ready-made-sqlite-files
# !ls ~/.data/oaklib

### Create an adapter to connect to the SQLite ontology database file

In [4]:
# Create a wrapper to connect to the mondo database
# If the ontology database file does not already exist in ~/.data/oaklib the ontolgy file will be downloaded.

# db_path = <PATH-TO-YOUR-DATABASE-FILE>
# db_path = "./ontology_downloads/mondo.db"
# adapter = get_adapter(f"sqlite:{db_path}")

adapter = get_adapter(f"sqlite:obo:mondo")

### Check the connection and get the ontology metadata

In [5]:
# Get ontology metadata
for ont in adapter.ontologies():
    ontology_metadata = adapter.ontology_metadata_map(ont)

# Display ontology metadata
display(ontology_metadata)

{'id': ['obo:mondo.owl'],
 'IAO:0000700': ['MONDO:0042489',
  'MONDO:0021178',
  'MONDO:0021125',
  'MONDO:0000001'],
 'dce:description': ['A semi-automatically constructed ontology that merges in multiple disease resources to yield a coherent merged ontology.'],
 'dce:title': ['Mondo Disease Ontology'],
 'dcterms:license': ['<http://creativecommons.org/licenses/by/4.0/>'],
 'dcterms:source': ['<https://rarediseases.info.nih.gov/>',
  '<https://id.nlm.nih.gov/mesh/>',
  '<http://www.orpha.net/ontology/orphanet.owl>',
  'obo:uberon.owl',
  'obo:ncit.owl',
  'obo:ncbitaxon.owl',
  'obo:mf.owl',
  'obo:hp.owl',
  'obo:go.owl',
  'obo:envo.owl',
  'obo:doid.owl',
  'obo:chebi.owl'],
 'foaf:homepage': ['http://obofoundry.org/ontology/mondo.html'],
 'oio:hasOBOFormatVersion': ['1.2'],
 'owl:versionIRI': ['obo:mondo/releases/2023-09-12/mondo.owl'],
 'rdf:type': ['owl:Ontology'],
 'rdfs:comment': ['Includes Ontology(OntologyID(Anonymous-11)) [Axioms: 74703 Logical Axioms: 0]'],
 'sh:prefix': [

In [ ]:
# Get all descendants for a CURIE
curie = "MONDO:0018076"

#NOTE: The input curie (e.g. "MONDO:0018076") is returned from this call
all_descendants = set(adapter.descendants([curie], predicates=[IS_A]))
display(len(all_descendants), all_descendants)


In [ ]:
# View results for CURIE in OLS for comparison
import webbrowser

# convert curie to purl
url = adapter.curie_to_uri(curie)

# open ols browser page for purl
webbrowser.open(url)

In [ ]:
# Get annotations for descendants

for child in all_descendants:
    print(child, adapter.label(child))

In [ ]:
# Get Ancestors
# TODO: Convert these to helper methods that can be called while traversing through list of parents or children

# for anc in db_adapter.ancestors("MONDO:0018076", predicates=[IS_A]):
#     print(anc, db_adapter.label(anc))

In [7]:
# Get a list of all Annotation properties used in the ontology

# for entity in adapter.entities(filter_obsoletes=True, owl_type = "owl:AnnotationProperty"):
#     display(entity)
# display(entity)